# FanDuel data

The purpose of this file is to take in an upcoming FanDuel competition (csv) and generate the inputs necessary for the ML model

In [18]:
import nbimporter
import pandas as pd
from datetime import datetime
import player_matches_data as pm_data
import games_data

Importing Jupyter notebook from games_data.ipynb


In [2]:
competition = pd.read_csv("../data/FanDuel_sample_competition.csv")

In [3]:
competition = competition[competition['Injury Indicator'] != "O"].copy()

In [4]:
date = datetime.today()
year = date.year
month = date.month
day = date.day

date = datetime(year, month, day) # strips precision

In [5]:
def add_home_game(row):
    return row['team_key'] == row['Game'].split("@")[1]

In [6]:
competition['date'] = date
competition['year'] = year
competition['month'] = month
competition['day'] = day
competition = competition.rename(
    columns={
        'Nickname': 'name',
        'Team': 'team_key',
        'Opponent': 'opponent_key'
    }
)
competition['home_game'] = competition.apply(add_home_game, axis = 1)

In [7]:
name_id_mapping = pd.read_csv("../data/name_id_mapping.csv")

In [8]:
def add_player_id(row):
    name = row['name']
    ids = name_id_mapping[name_id_mapping.name == name]['player_id']
    
    if len(ids) > 0:
        return ids.iloc[0]
    else:
        print("Please update player_id for name " + name)
        return None

In [9]:
competition['player_id'] = competition.apply(add_player_id, axis = 1)

Please update player_id for name Norvel Pelle
Please update player_id for name Thanasis Antetokounmpo
Please update player_id for name Jalen McDaniels
Please update player_id for name Michael Frazier
Please update player_id for name Robert Franks
Please update player_id for name Charlie Brown
Please update player_id for name Brian Bowen
Please update player_id for name Brandon Goodwin
Please update player_id for name Marial Shayok


In [10]:
fd_complete = pm_data.add_game_ids(competition)
fd_complete = pm_data.add_season_start_year(fd_complete)

In [11]:
fd_complete

,game_id,Id,Position,First Name,name,season_start_year,Last Name,FPPG,Played,Salary,...,Injury Details,Tier,Unnamed: 14,Unnamed: 15,date,year,month,day,home_game,player_id
0,20191130_MIL_CHA,41124-40199,SF,Giannis,Giannis Antetokounmpo,2019,Antetokounmpo,61.242104,19,12400,...,Back,NaN,NaN,NaN,2019-11-30,2019,11,30,True,antetgi01
1,20191130_HOU_ATL,41124-9644,SG,James,James Harden,2019,Harden,57.861111,18,11500,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,True,hardeja01
2,20191130_PHI_IND,41124-49111,C,Joel,Joel Embiid,2019,Embiid,44.980001,15,10400,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,True,embiijo01
3,20191130_HOU_ATL,41124-84671,PG,Trae,Trae Young,2019,Young,45.411112,18,9800,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,False,youngtr01
4,20191130_HOU_ATL,41124-9642,PG,Russell,Russell Westbrook,2019,Westbrook,43.181252,16,9000,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,True,westbru01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,20191130_PHI_IND,41124-17384,C,Kyle,Kyle O'Quinn,2019,O'Quinn,10.392308,13,3500,...,Calf,NaN,NaN,NaN,2019-11-30,2019,11,30,True,oquinky01
97,20191130_PHI_IND,41124-110829,SG,Brian,Brian Bowen,2019,Bowen,2.066667,3,3500,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,False,None
98,20191130_PHI_IND,41124-84718,PF,Alize,Alize Johnson,2019,Johnson,4.200000,7,3500,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,False,johnsal02
99,20191130_HOU_ATL,41124-96763,PG,Brandon,Brandon Goodwin,2019,Goodwin,2.787500,16,3500,...,NaN,NaN,NaN,NaN,2019-11-30,2019,11,30,False,None


In [12]:
stats = ['secs', 'fgm', 'fga', '3pm', '3pa', 'ftm', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tvr', 'pf', 'fp']

In [13]:
for stat in stats:
    fd_complete[stat] = 0

In [14]:
fd_complete

,game_id,Id,Position,First Name,name,season_start_year,Last Name,FPPG,Played,Salary,...,ftm,fta,orb,drb,ast,stl,blk,tvr,pf,fp
0,20191130_MIL_CHA,41124-40199,SF,Giannis,Giannis Antetokounmpo,2019,Antetokounmpo,61.242104,19,12400,...,0,0,0,0,0,0,0,0,0,0
1,20191130_HOU_ATL,41124-9644,SG,James,James Harden,2019,Harden,57.861111,18,11500,...,0,0,0,0,0,0,0,0,0,0
2,20191130_PHI_IND,41124-49111,C,Joel,Joel Embiid,2019,Embiid,44.980001,15,10400,...,0,0,0,0,0,0,0,0,0,0
3,20191130_HOU_ATL,41124-84671,PG,Trae,Trae Young,2019,Young,45.411112,18,9800,...,0,0,0,0,0,0,0,0,0,0
4,20191130_HOU_ATL,41124-9642,PG,Russell,Russell Westbrook,2019,Westbrook,43.181252,16,9000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,20191130_PHI_IND,41124-17384,C,Kyle,Kyle O'Quinn,2019,O'Quinn,10.392308,13,3500,...,0,0,0,0,0,0,0,0,0,0
97,20191130_PHI_IND,41124-110829,SG,Brian,Brian Bowen,2019,Bowen,2.066667,3,3500,...,0,0,0,0,0,0,0,0,0,0
98,20191130_PHI_IND,41124-84718,PF,Alize,Alize Johnson,2019,Johnson,4.200000,7,3500,...,0,0,0,0,0,0,0,0,0,0
99,20191130_HOU_ATL,41124-96763,PG,Brandon,Brandon Goodwin,2019,Goodwin,2.787500,16,3500,...,0,0,0,0,0,0,0,0,0,0


In [15]:
old_data = pd.read_csv("../data/20100101_20191118_player_matches.csv")

ROLLING_GAMES_WINDOW = 5

In [16]:
fd_complete = pm_data.roll_with_old_data(fd_complete, old_data, stats, ROLLING_GAMES_WINDOW)

## Create games dataframe (input to model)

In [19]:
PLAYER_SEASONS_FILENAME = "../data/2009_2019_player_seasons_unique.csv"
player_seasons = pd.read_csv(PLAYER_SEASONS_FILENAME)

In [22]:
per_game_df = games_data.create_games_data(fd_complete, player_seasons, 7, ROLLING_GAMES_WINDOW)

In [23]:
per_game_df

,player_id_p1,player_id_p2,player_id_p3,player_id_p4,player_id_p5,player_id_p6,player_id_p7,player_id_p8,player_id_p9,player_id_p10,...,fp_p23,fp_p24,fp_p25,fp_p26,fp_p27,fp_p28,fp_p29,fp_p30,fp_p31,fp_p32
0,hardeja01,youngtr01,westbru01,capelca01,parkeja01,houseda01,tuckepj01,huntede01,jonesda03,bembrde01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,antetgi01,bledser01,grahade01,roziete01,middlkh01,bridgmi02,washipj01,zelleco01,biyombi01,divindo01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,embiijo01,simmobe01,brogdma01,sabondo01,horfoal01,harrito02,lambje01,turnemy01,richajo01,warretj01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Predict on model